In [ ]:
! sudo apt update
! sudo apt-get install -y fonts-dejavu
! sudo apt install tesseract-ocr
! sudo apt install libtesseract-dev
! pip install pytesseract

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,200 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,639 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

# Add WaterMark in Image

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np

def embed_watermark_with_text(original_image_path, text, output_image_path):
    original = Image.open(original_image_path).convert("RGB")

    watermark = Image.new("L", original.size, 0)
    draw = ImageDraw.Draw(watermark)

    font_size = int(min(original.size) * 0.04)  
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", font_size) 

    text_width, text_height = draw.textbbox((0, 0), text, font=font)[2:]
    for y in range(0, watermark.height, text_height + 10): 
        for x in range(0, watermark.width, text_width + 30): 
            draw.text((x, y), text, fill=255, font=font)

    original_array = np.array(original)
    watermark_array = np.array(watermark) // 255 

    watermarked_array = original_array.copy()
    watermarked_array[:, :, 2] = (original_array[:, :, 2] & 0xFE) | watermark_array

    watermarked_image = Image.fromarray(watermarked_array)
    watermarked_image.save(output_image_path)
    print(f"Watermarked image saved to {output_image_path}")
original_image_path = "/content/wallpaper.jpg"
text = "crowdgenai-d8s1h8s6g"
output_image_path = "watermarked_image.jpg"

embed_watermark_with_text(original_image_path, text, output_image_path)

Watermarked image saved to watermarked_image.jpg


# Show WaterMark

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np

def reveal_watermark(watermarked_image_path, output_revealed_path):
    watermarked = Image.open(watermarked_image_path).convert("RGB")
    watermarked_array = np.array(watermarked)

    watermark_array = (watermarked_array[:, :, 2] & 1) * 255

    alpha = 0.5 
    revealed_array = watermarked_array.copy()
    revealed_array[:, :, 2] = ((watermarked_array[:, :, 2] & 0xFE) | ((watermark_array // 255) * 127))

    revealed_image = Image.fromarray(revealed_array.astype(np.uint8))
    revealed_image.save(output_revealed_path)
    print(f"Revealed watermark saved to {output_revealed_path}")

output_image_path = "/content/invisible2.png"
output_revealed_path = "revealed_watermark.png"

reveal_watermark(output_image_path, output_revealed_path)

Revealed watermark saved to revealed_watermark.png


# Watermark detection

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np

def check_watermark(image_path, expected_text):
    image = Image.open(image_path).convert("RGB")
    image_array = np.array(image)
    watermark_array = (image_array[:, :, 2] & 1) * 255

    watermark_image = Image.fromarray(watermark_array.astype(np.uint8))

    try:
        import pytesseract
        detected_text = pytesseract.image_to_string(watermark_image, config="--psm 6").strip()
        if expected_text in detected_text:
            print("Watermark detected with matching text.")
            return True
        else:
            print("Watermark detected but text does not match.")
            return False
    except ImportError:
        print("pytesseract is required for text detection but is not installed.")
        return False

image_to_check = "/content/invisible1.png"
check_watermark(image_to_check, "crowdgen")

Watermark detected with matching text.


True